In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(False)

In [2]:
client = MongoClient()
loladb = client.loladb73
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

2829
30993


In [3]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

In [4]:
def AddToDict(a_dict, a_key, a_value):
    if a_key in a_dict.keys():
        a_dict[a_key] += a_value
    else:
        a_dict[a_key] = a_value

In [35]:
champions_records = []

for a_idx, a_match in enumerate(matches_collection.find()):
    #check the champs appear in this game
    if a_match['matchDuration'] <= 1200:
        continue
        
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue
        
    participants = a_match['participants']
    blue_champions = []
    red_champions = []
    for a_participant in participants:
        champion_id = a_participant['championId']
        champion_index = a_participant['participantId']
        champion_side = 0
        if champion_index > 5:
            champion_side = 1
            red_champions.append(champion_id)
        else:
            blue_champions.append(champion_id)
            
    a_record = {}
    a_record['red'] = red_champions
    a_record['blue'] = blue_champions
    a_record['winside'] = winning_side
    champions_records.append(a_record)
            
    if a_idx % 1000 == 0:
        print a_idx

0
1000
3000
4000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
28000
29000
30000


In [15]:
print len(champions_records)
print champions_records[0]

28327
{'blue': array([ 21,  42,  60,  81, 126]), 'red': array([ 58,  90, 121, 202, 245])}


In [36]:
for a_idx, a_record in enumerate(champions_records):
    a_record['red'] = np.sort(a_record['red'])
    a_record['blue'] = np.sort(a_record['blue'])
    if a_idx % 1000 == 0:
        print a_idx

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [17]:
def GetSameNumber(A, B):
    #assume both A and B has 5 elements
    idx_a = 0
    idx_b = 0
    same_number = 0
    while idx_a < 5 and idx_b < 5:
        if A[idx_a] == B[idx_b]:
            same_number += 1
            idx_a += 1
            idx_b += 1
        elif A[idx_a] < B[idx_b]:
            idx_a += 1
        else:
            idx_b += 1
    return same_number

In [42]:
record_num = len(champions_records)
counter = 0
max_same = 8
max_pair = []
for i in range(record_num):
    for j in range(record_num):
        if i >= j:
            continue
        a_same = GetSameNumber(champions_records[i]['blue'], champions_records[j]['blue']) + GetSameNumber(champions_records[i]['red'], champions_records[j]['red'])
        if a_same >= max_same:
            #max_same = a_same
            max_pair.append((i, j))
        counter += 1
        if counter % 1000000 == 0:
            print counter
        if max_same == 10:
            break


1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
11100000

In [25]:
print record_num * (record_num - 1) / 2

401195301


In [44]:
print max_pair

[(94, 17744), (113, 20340), (249, 3890), (428, 22768), (435, 1959), (517, 14891), (517, 28178), (637, 26130), (691, 2262), (729, 2943), (805, 1122), (941, 19504), (1016, 1237), (1189, 18901), (1213, 1245), (1213, 6877), (1215, 24308), (1229, 21856), (1556, 3827), (1609, 27217), (1680, 3567), (1704, 26794), (1740, 3992), (1747, 9335), (1824, 8904), (2118, 2177), (2260, 20827), (2330, 2967), (2334, 17255), (2430, 18943), (2584, 17990), (2747, 3744), (2911, 19435), (2931, 10920), (3052, 13225), (3052, 19313), (3166, 11725), (3166, 22312), (3173, 3575), (3181, 6885), (3248, 15203), (4164, 13264), (4321, 21114), (4676, 5096), (6528, 15167), (6818, 16040), (8178, 19794), (8242, 27669), (8599, 23916), (9027, 23017), (10552, 27212), (10673, 21155), (10903, 19832), (11659, 15985), (11702, 20235), (12328, 14966), (12423, 19038), (12660, 19507), (12912, 28274), (13282, 25408), (13842, 20588), (14273, 20329), (15167, 16589), (15981, 23878), (17455, 18061), (18277, 23652), (20809, 21927), (21139, 2

In [33]:
print max_pair
print champions_records[max_pair[0]]
print champions_records[max_pair[1]]

[3248, 15203]
{'blue': array([ 13,  25, 110, 114, 121]), 'red': array([ 42,  57,  81, 104, 412])}
{'blue': array([ 13,  53, 110, 114, 121]), 'red': array([ 42,  57,  81, 104, 412])}


In [48]:
for a_pair in max_pair:
    print "+++++"
    PrintNames(champions_records[a_pair[0]]['blue'])
    PrintNames(champions_records[a_pair[0]]['red'])
    print champions_records[a_pair[0]]['winside']
    print "VS"
    PrintNames(champions_records[a_pair[1]]['blue'])
    PrintNames(champions_records[a_pair[1]]['red'])
    print champions_records[a_pair[1]]['winside']
    print "-----"

+++++
Maokai LeeSin Ezreal Talon Thresh
Corki Blitzcrank Rengar Syndra Lucian
0
VS
Maokai LeeSin Ezreal Talon Thresh
Olaf Corki Blitzcrank Graves Lucian
0
-----
+++++
Leblanc Maokai LeeSin Ezreal Thresh
Corki Blitzcrank Graves Varus Ekko
1
VS
Leblanc LeeSin Poppy Ezreal Thresh
Blitzcrank Graves Varus Zac Ekko
1
-----
+++++
Karma Maokai Orianna Graves Varus
Corki LeeSin Camille Jhin Thresh
1
VS
Irelia Karma Orianna Graves Varus
Corki Renekton LeeSin Jhin Thresh
0
-----
+++++
LeeSin Malzahar Varus Nautilus Zed
Corki Blitzcrank Maokai Khazix Jhin
1
VS
Alistar LeeSin Varus Nautilus Zed
Soraka Corki Maokai Khazix Jhin
0
-----
+++++
Ashe Corki Karma Graves Jayce
Renekton LeeSin Diana Syndra Jhin
0
VS
Ashe Corki Karma Graves Jayce
Renekton LeeSin Malzahar Varus Jhin
0
-----
+++++
Anivia LeeSin Shen Jhin Thresh
Blitzcrank Akali Varus Khazix Zed
1
VS
Singed Anivia LeeSin Jhin Thresh
Blitzcrank Akali Varus Khazix Jayce
1
-----
+++++
Anivia LeeSin Shen Jhin Thresh
Blitzcrank Akali Varus Khazix Ze

In [46]:
def PrintNames(team):
    print GetNameFromId(team[0]),GetNameFromId(team[1]),GetNameFromId(team[2]),GetNameFromId(team[3]),GetNameFromId(team[4])

In [52]:
print len(max_pair)
dict00 = 0
dict01 = 0
dict10 = 0
dict11 = 0
for a_pair in max_pair:
    wina = champions_records[a_pair[0]]['winside']
    winb = champions_records[a_pair[1]]['winside']
    if wina == 0 and winb == 0:
        dict00 += 1
    elif wina == 0 and winb == 1:
        dict01 += 1
    elif wina == 1 and winb == 0:
        dict10 += 1
    else:
        dict11 += 1
print dict00, dict01, dict10, dict11

70
13 15 22 20


In [41]:
print GetNameFromId(25)
print GetNameFromId(53)

Morgana
Blitzcrank
